<a href="https://colab.research.google.com/github/SohilaOsama/eicu-code/blob/master/Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from scipy import stats
from scipy.stats import norm, skew, boxcox
from collections import Counter
from numpy import mean
from numpy import std
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, plot_confusion_matrix, auc
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

#XGBOOST
from xgboost import XGBClassifier

In [15]:
data = pd.read_csv("/content/data01.csv")

In [16]:
print("Data Columns --> ",data.columns)

Data Columns -->  Index(['group', 'ID', 'outcome', 'age', 'gendera', 'BMI', 'hypertensive',
       'atrialfibrillation', 'CHD with no MI', 'diabetes', 'deficiencyanemias',
       'depression', 'Hyperlipemia', 'Renal failure', 'COPD', 'heart rate',
       'Systolic blood pressure', 'Diastolic blood pressure',
       'Respiratory rate', 'temperature', 'SP O2', 'Urine output',
       'hematocrit', 'RBC', 'MCH', 'MCHC', 'MCV', 'RDW', 'Leucocyte',
       'Platelets', 'Neutrophils', 'Basophils', 'Lymphocyte', 'PT', 'INR',
       'NT-proBNP', 'Creatine kinase', 'Creatinine', 'Urea nitrogen',
       'glucose', 'Blood potassium', 'Blood sodium', 'Blood calcium',
       'Chloride', 'Anion gap', 'Magnesium ion', 'PH', 'Bicarbonate',
       'Lactic acid', 'PCO2', 'EF'],
      dtype='object')


In [17]:
data.drop(['group', 'ID'], axis = 1, inplace = True)

In [18]:
['group', 'ID','gendera', 'BMI', 'hypertensive',
'atrialfibrillation', 'CHD with no MI', 'diabetes', 'deficiencyanemias',
'depression', 'Hyperlipemia', 'Renal failure', 'COPD','Systolic blood pressure', 'temperature'
,'SP O2','hematocrit','RBC','MCHC','Leucocyte','PT', 'INR','Creatinine','glucose','Chloride','Magnesium ion',
 'PH', 'Bicarbonate','PCO2', 'EF']

['group',
 'ID',
 'gendera',
 'BMI',
 'hypertensive',
 'atrialfibrillation',
 'CHD with no MI',
 'diabetes',
 'deficiencyanemias',
 'depression',
 'Hyperlipemia',
 'Renal failure',
 'COPD',
 'Systolic blood pressure',
 'temperature',
 'SP O2',
 'hematocrit',
 'RBC',
 'MCHC',
 'Leucocyte',
 'PT',
 'INR',
 'Creatinine',
 'glucose',
 'Chloride',
 'Magnesium ion',
 'PH',
 'Bicarbonate',
 'PCO2',
 'EF']

In [59]:
data['BMI'].fillna(data['BMI'].mean(), inplace = True)


data['heart rate'].fillna(data['heart rate'].mean(), inplace = True)
data['Systolic blood pressure'].fillna(data['Systolic blood pressure'].mean(), inplace = True)
data['Diastolic blood pressure'].fillna(data['Diastolic blood pressure'].mean(), inplace = True)
data['Respiratory rate'].fillna(data['Respiratory rate'].mean(), inplace = True)
data['SP O2'].fillna(data['SP O2'].mean(), inplace = True)
data['Urine output'].fillna(data['Urine output'].mean(), inplace = True)
data['Neutrophils'].fillna(data['Neutrophils'].mean(), inplace = True)
data['Basophils'].fillna(data['Basophils'].mean(), inplace = True)
data['Lymphocyte'].fillna(data['Lymphocyte'].mean(), inplace = True)
data['PT'].fillna(data['PT'].mean(), inplace = True)
data['INR'].fillna(data['INR'].mean(), inplace = True)
data['Creatine kinase'].fillna(data['Creatine kinase'].mean(), inplace = True)
data['glucose'].fillna(data['glucose'].mean(), inplace = True)
data['Blood calcium'].fillna(data['Blood calcium'].mean(), inplace = True)
data['PH'].fillna(data['PH'].mean(), inplace = True)
data['Lactic acid'].fillna(data['Lactic acid'].mean(), inplace = True)
data['PCO2'].fillna(data['PCO2'].mean(), inplace = True)
data['temperature'].fillna(data['temperature'].mean(), inplace = True)

print(data.isnull().sum())

outcome                     0
age                         0
gendera                     0
BMI                         0
hypertensive                0
atrialfibrillation          0
CHD with no MI              0
diabetes                    0
deficiencyanemias           0
depression                  0
Hyperlipemia                0
Renal failure               0
COPD                        0
heart rate                  0
Systolic blood pressure     0
Diastolic blood pressure    0
Respiratory rate            0
temperature                 0
SP O2                       0
Urine output                0
hematocrit                  0
RBC                         0
MCH                         0
MCHC                        0
MCV                         0
RDW                         0
Leucocyte                   0
Platelets                   0
Neutrophils                 0
Basophils                   0
Lymphocyte                  0
PT                          0
INR                         0
NT-proBNP 

In [19]:
#data.drop(['group', 'ID','gendera', 'BMI', 'hypertensive',
#'atrialfibrillation', 'CHD with no MI', 'diabetes', 'deficiencyanemias',
#'depression', 'Hyperlipemia', 'Renal failure', 'COPD','Systolic blood pressure', 'temperature'
#,'SP O2','hematocrit','RBC','MCHC','Leucocyte','PT', 'INR','Creatinine','glucose','Chloride','Magnesium ion',
 #'PH', 'Bicarbonate','PCO2', 'EF'], axis = 1, inplace = True)

In [38]:
feature_cols = ['Anion gap','Lactic acid','Blood calcium','Lymphocyte','Leucocyte','heart rate',
'Blood sodium','Urine output','Platelets','Urea nitrogen','age','MCH','RBC','Creatinine','PCO2',
'Blood potassium','Respiratory rate','MCHC']

In [39]:
X = data[feature_cols] # Features
y = data.outcome

In [51]:
X['Lactic acid'].fillna(X['Lactic acid'].median(), inplace = True)
X['Blood calcium'].fillna(X['Blood calcium'].mean(), inplace = True)
X['Lymphocyte'].fillna(X['Lymphocyte'].median(), inplace = True)
X['Urine output'].fillna(X['Urine output'].median(), inplace = True)
X['PCO2'].fillna(X['PCO2'].mean(), inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [52]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [62]:
X_train['Lactic acid'].fillna(X_train['Lactic acid'].median(), inplace = True)
X_train['Blood calcium'].fillna(X_train['Blood calcium'].mean(), inplace = True)
X_train['Lymphocyte'].fillna(X_train['Lymphocyte'].median(), inplace = True)
X_train['Urine output'].fillna(X_train['Urine output'].median(), inplace = True)
X_train['PCO2'].fillna(X_train['PCO2'].mean(), inplace = True)
X_train['heart rate'].fillna(X_train['heart rate'].mean(), inplace = True)
X_train['Respiratory rate'].fillna(X_train['Respiratory rate'].mean(), inplace = True)

In [63]:
print(X_train.isna().sum())

Anion gap           0
Lactic acid         0
Blood calcium       0
Lymphocyte          0
Leucocyte           0
heart rate          0
Blood sodium        0
Urine output        0
Platelets           0
Urea nitrogen       0
age                 0
MCH                 0
RBC                 0
Creatinine          0
PCO2                0
Blood potassium     0
Respiratory rate    0
MCHC                0
dtype: int64


In [64]:
X_test['Lactic acid'].fillna(X_test['Lactic acid'].median(), inplace = True)
X_test['Blood calcium'].fillna(X_test['Blood calcium'].mean(), inplace = True)
X_test['Lymphocyte'].fillna(X_test['Lymphocyte'].median(), inplace = True)
X_test['Urine output'].fillna(X_test['Urine output'].median(), inplace = True)
X_test['PCO2'].fillna(X_test['PCO2'].mean(), inplace = True)
X_test['heart rate'].fillna(X_test['heart rate'].mean(), inplace = True)
X_test['Respiratory rate'].fillna(X_test['Respiratory rate'].mean(), inplace = True)

In [65]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()



In [66]:
print(y_train.isna().sum())

0


In [67]:
# fit the model with data
logreg.fit(X_train,y_train)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [68]:
y_pred=logreg.predict(X_test)

In [69]:
# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[294,   2],
       [ 52,   6]])

In [70]:
# import required modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [72]:
print("Precision:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 0.847457627118644
Accuracy: 0.75
Recall: 0.10344827586206896
